In [30]:
import os
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [31]:
#pred_csv = 'work_dirs/inseo_test/yolox_x(scaling+soft nms)/val_best_bbox_mAP_50_epoch_42.csv'
pred_csv = 'ensemble_2.csv'

#pred_csv = 'work_dirs/inseo_test/yolox_x(scaling+soft nms)/val_finding_best_bbox_mAP_50_epoch_42.csv'

pred_csv = pd.read_csv(pred_csv)

#class_csv = 'classification/work_dirs/classification/val_class.csv'
class_csv = 'classification/work_dirs/classification/test_class.csv'
class_csv = pd.read_csv(class_csv)

In [32]:
# csv_modify
df = pred_csv.copy()
for i in range(len(pred_csv)):
    preds,image_id = pred_csv.loc[i]
    classes = class_csv[class_csv['image_id']==image_id]['classe_str'][i]
    
    preds = list(map(float,preds.split()))
    classes = list(map(float,classes.split()))
    
    #print(len(pred),len(classes))
    assert len(preds)%6 == 0 and len(classes)%10==0
    
    tmp = list(map(float,pred_csv.loc[i]['PredictionString'].split()))
    
    #print('before :',tmp)
    for j in range(0,len(tmp),6):
        tmp[j] = int(tmp[j])
        tmp[j+1] = tmp[j+1]*(classes[tmp[j]]**(0.22))
    tmp = " ".join(map(str,tmp))+' '
    df.loc[i,'PredictionString'] = tmp

In [33]:
pred_csv.tail()

,PredictionString,image_id
4866,1 0.5080580711364746 291.42108154296875 346.31...,test/4866.jpg
4867,3 0.7856745719909668 414.3136291503906 315.421...,test/4867.jpg
4868,7 0.55253666639328 113.62055206298828 465.3815...,test/4868.jpg
4869,7 0.41058045625686646 2.180711030960083 235.13...,test/4869.jpg
4870,4 0.655908465385437 371.68658447265625 204.432...,test/4870.jpg


In [34]:
df.tail()

,PredictionString,image_id
4866,1 0.4526106719406756 291.42108154296875 346.31...,test/4866.jpg
4867,3 0.7427717204613102 414.3136291503906 315.421...,test/4867.jpg
4868,7 0.5174714835957361 113.62055206298828 465.38...,test/4868.jpg
4869,7 0.20197696795674425 2.180711030960083 235.13...,test/4869.jpg
4870,4 0.6558949295631684 371.68658447265625 204.43...,test/4870.jpg


In [35]:
df.to_csv(os.path.join('calibration_ensemble_2.csv'), index=None)

# grid search

In [5]:
GT_JSON = '../dataset/kfold/seed41/val_4.json'
#PRED_CSV = '../../sample_submission/train_sample.csv'
LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal", 
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

gt = []

coco = COCO(GT_JSON)

for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                   float(annotation['bbox'][0]),
                   float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                   float(annotation['bbox'][1]),
                   (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [6]:
def create_new_pred(pred_df):
    new_pred = []

    file_names = pred_df['image_id'].values.tolist()
    bboxes = pred_df['PredictionString'].values.tolist()

    '''
    create new_pred
    '''

    for i, bbox in enumerate(bboxes):
        if isinstance(bbox, float):
            print(f'{file_names[i]} empty box')

    for file_name, bbox in zip(file_names, bboxes):
        boxes = np.array(str(bbox).split(' '))

        if len(boxes) % 6 == 1:
            boxes = boxes[:-1].reshape(-1, 6)
        elif len(boxes) % 6 == 0:
            boxes = boxes.reshape(-1, 6)
        else:
            raise Exception('error', 'invalid box count')
        for box in boxes:
            new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])
    return new_pred

In [14]:
def calibration(c):
    global gt
    df = pred_csv.copy()
    for i in range(len(pred_csv)):
        preds,image_id = pred_csv.loc[i]
        classes = class_csv[class_csv['image_id']==image_id]['classe_str'][i]

        preds = list(map(float,preds.split()))
        classes = list(map(float,classes.split()))

        #print(len(pred),len(classes))
        assert len(preds)%6 == 0 and len(classes)%10==0

        tmp = list(map(float,pred_csv.loc[i]['PredictionString'].split()))

        #print('before :',tmp)
        for j in range(0,len(tmp),6):
            tmp[j] = int(tmp[j])
            tmp[j+1] = tmp[j+1]*(classes[tmp[j]]**(c))
        tmp = " ".join(map(str,tmp))+' '
        df.loc[i,'PredictionString'] = tmp

    new_pred = create_new_pred(df)
    mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5,verbose=False)
    return mean_ap

In [17]:
max_c,max_score = 0, 0
for i in tqdm(range(0,100)):
    score = calibration(i/100)
    
    if max_score<score:
        max_score=score
        max_c=i/100
        print(max_c,max_score)

  1%|          | 1/100 [00:02<04:55,  2.99s/it]

0.0 0.524121973475698


  2%|▏         | 2/100 [00:05<04:52,  2.99s/it]

0.01 0.5278334912241285


  3%|▎         | 3/100 [00:08<04:48,  2.98s/it]

...

 21%|██        | 21/100 [01:02<03:57,  3.00s/it]

0.2 0.5347786720107777


 23%|██▎       | 23/100 [01:09<03:54,  3.04s/it]

0.22 0.5349084907570599


100%|██████████| 100/100 [05:00<00:00,  3.01s/it]


In [221]:
print(max_c,max_score)

0.0 0.5337842485825756
